In [4]:
import os
import json
import re
# imports
import javalang
import os
from app.database import Database
from app.classes import IterationStatus, BatchAttemptStatus, TestStatus, AttemptStatus,Dataset
from typing import List
from app.constants import CONFIG
import pickle
import xml.etree.ElementTree as ET
from app.utils import etree_to_dict
from tqdm import tqdm
from app.constants import CoverageMode
import pickle
from app.classes import Dataset
from app.constants import CONFIG

DATASET = "gson"
OUTPUT_DIR = fr"C:\Users\tianfy\Desktop\毕设\数据集\4.0\{DATASET}"


In [2]:
# read dataset
db = Database()
all_dataset = db.get_all()

read time: 0.04800009727478027
decompress time: 3.5530006885528564
loads time: 3.974311113357544
assembling time: 3.818988561630249


In [3]:
for b in all_dataset:
    print(b.dataset, b.doc_id)

lang_1_fixed 47
commons-csv 58
commons-cli 61
gson 66
jfreechart 82
lang_1_fixed 89
gson 98
commons-csv 99
commons-cli 100
jfreechart 101
jfreechart154 123
jfreechart154 126
lang_1_fixed 127
gson 128
jfreechart154 130


In [6]:
batch = [x for x in all_dataset if x.doc_id == 130][0]
attempts = batch.attempts
get_pass_attempts = lambda attempts: [x for x in attempts if x.type == IterationStatus.Type.PASS]
def get_first_attempts(attempts: List[AttemptStatus]):
    first_attempts = []
    for a in attempts:
        cpy_a = a.model_copy(deep=True)
        if cpy_a.type != IterationStatus.Type.PASS:
            first_attempts.append(cpy_a)
            continue
        if not cpy_a.sub_iteration_status:
            pass
        else:
            it: IterationStatus = cpy_a.sub_iteration_status[0]
            cpy_a.sub_iteration_status = []
            cpy_a.iteration_status = None
            if not it.sub_test_status:
                assert False
                pass
            else:
                ts = it.sub_test_status[0]
                it.sub_test_status = []
                it.test_status = None
                it.add_test_status(ts)
                if ts.type == TestStatus.Type.ERROR:
                    it.type = IterationStatus.Type.COMPILE_ERROR
                else:
                    it.type = IterationStatus.Type.PASS if it.all_pass else IterationStatus.Type.RUNTIME_ERROR
            cpy_a.add_iteration_status(it)
        first_attempts.append(cpy_a)
    return first_attempts
from tqdm import tqdm
def check_is_valid(flow_run_id: str):
    import requests
    base_url = "http://127.0.0.1:4200/api/logs/filter"
    payload = {
        "logs": {
            "level": {
                "ge_": 30
            },
            "flow_run_id": {
                "any_": [
                    f"{flow_run_id}"
                ]
            }
        },
        "sort": "TIMESTAMP_ASC",
        "offset": 0,
        "limit": 200
    }

    # parse response
    response = requests.post(base_url, json=payload)
    arr = response.json()
    messages = [x['message'] for x in arr]
    for i in range(len(messages) - 1):
        if 'source folder D' in messages[i]:
            if 'fix compile error' in messages[i + 1]:
                return False
            else:
                return True
    return True
def first_attempt_filter(attempts: List[AttemptStatus]):
    filtered = []
    for a in tqdm(attempts):
        if a.type == IterationStatus.Type.PASS and check_is_valid(a.link.split("/")[-1]):
            filtered.append(a)
    print(f"filtered: {len(filtered)}")
    return filtered
def get_iteration_n_attempts(n):
    iteration_attempts = []
    for a in attempts:
        cpy_a = a.model_copy(deep=True)

        if cpy_a.index < n:
            iteration_attempts.append(cpy_a)
            continue

        sub_its = cpy_a.sub_iteration_status
        cpy_a.sub_iteration_status = []
        cpy_a.iteration_status = None

        count = 0
        for it in sub_its:
            cpy_a.add_iteration_status(it)
            count += 1
            if count == n:
                break

        iteration_attempts.append(cpy_a)
        try:
            assert len(cpy_a.sub_iteration_status) == n
        except:
            print(cpy_a.idx, len(cpy_a.sub_iteration_status))
    return iteration_attempts


pass_attempts: list[AttemptStatus] = get_pass_attempts(attempts)
print("pass attempts", len(pass_attempts))

pass attempts 2593


In [7]:
def replace_class_name(code:str, key):
    pattern = "public class ([\w]+)"
    class_name = re.search(pattern, code).group(1).strip()
    code = code.replace(class_name, key)
    return code
def get_package_path(code: str):
    match = re.search("package ([\w.]+);", code)
    if match:
        return match.group(1).replace(".", "/")
    else:
        assert False, "no package found"
from tqdm import tqdm
import shutil
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

for attempt in tqdm(pass_attempts):
    method = attempt.method.split("(")[0]
    key = f"{attempt.clazz}_{method}_{attempt.idx}_Test"
    code = replace_class_name(attempt.test_code, key)
    package_path = get_package_path(code)
    OUTPUT_PATH = f"{OUTPUT_DIR}/{package_path}"
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    with open(os.path.join(OUTPUT_PATH, f"{key}.java"), "w", encoding="utf8") as f:
        f.write(code)


100%|██████████| 2593/2593 [00:01<00:00, 1635.27it/s]


In [8]:
import zipfile

EXECUTION_PATH = fr"D:\IDEA_Projects\TestJavaCode\src\test\java"
SOURCE_PATH = fr"D:\IDEA_Projects\TestJavaCode\src\main"
PROJECT_PATH = fr"D:\IDEA_Projects\TestJavaCode"
if os.path.exists(EXECUTION_PATH):
    shutil.rmtree(EXECUTION_PATH)
if os.path.exists(SOURCE_PATH):
    shutil.rmtree(SOURCE_PATH)
print("clean up done")

clean up done


In [9]:
shutil.copytree(OUTPUT_DIR, EXECUTION_PATH)
dataset_path = f"{CONFIG.dataset_folder}/{DATASET}.zip"
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(PROJECT_PATH)
print("extract done")

extract done


In [38]:
import os
import subprocess
import psutil
import time

def _run_command(command: str):
    def kill_proc_tree(pid, including_parent=True):
        try:
            parent = psutil.Process(pid)
        except psutil.NoSuchProcess:
            return # 该进程已经不存在了
        children = parent.children(recursive=True)
        for child in children:
            child.kill()
        _, still_alive = psutil.wait_procs(children, timeout=5)
        if including_parent:
            parent.kill()
            parent.wait(5)

    os.environ["JAVA_TOOL_OPTIONS"] = "-Duser.language=en"
    process = subprocess.Popen(command,
                               cwd=PROJECT_PATH,
                               shell=True,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE)

    try:
        stdout, stderr = process.communicate(timeout=180)
    except subprocess.TimeoutExpired as e:
        kill_proc_tree(process.pid)
        process.kill()  # 确保杀死主进程
        process.wait()  # 等待进程退出
        raise e

    print(stdout)
_run_command("mvn clover:setup test clover:aggregate clover:clover -Djava.awt.headless=true")


b"[INFO] Scanning for projects...\r\n[WARNING] \r\n[WARNING] Some problems were encountered while building the effective model for org.example:TestJavaCode:jar:1.0-SNAPSHOT\r\n[WARNING] 'build.plugins.plugin.version' for org.apache.maven.plugins:maven-surefire-plugin is missing. @ line 90, column 21\r\n[WARNING] \r\n[WARNING] It is highly recommended to fix these problems because they threaten the stability of your build.\r\n[WARNING] \r\n[WARNING] For this reason, future Maven versions might no longer support building such malformed projects.\r\n[WARNING] \r\n[INFO] \r\n[INFO] ----------------------< org.example:TestJavaCode >----------------------\r\n[INFO] Building TestJavaCode 1.0-SNAPSHOT\r\n[INFO]   from pom.xml\r\n[INFO] --------------------------------[ jar ]---------------------------------\r\n[INFO] \r\n[INFO] --- clover:4.5.1:setup (default-cli) @ TestJavaCode ---\r\n[INFO] OpenClover Version 4.5.1, built on 2023-10-29\r\n[INFO] Updating existing database at 'D:\\IDEA_Projec

In [13]:
CLOVER_XML = fr"{PROJECT_PATH}/target/site/clover/clover.xml"
TARGET_CLVOER_XML = f"{os.path.dirname(OUTPUT_DIR)}/{DATASET}.xml"
shutil.copy(CLOVER_XML, TARGET_CLVOER_XML)


'C:\\Users\\tianfy\\Desktop\\毕设\\数据集\\4.0/jfreechart154.xml'

In [14]:
# read dataset
with open(f"{CONFIG.dataset_folder}/{DATASET}.pkl", "rb") as f:
    raw_dataset = pickle.load(f)
    dataset = Dataset(**raw_dataset)

In [15]:
from app.classes import IterationStatus

total_branch_tupe = (0, 0)
total_line_tupe = (0, 0)
count = 0

tree = ET.parse(CLOVER_XML)
root = tree.getroot()
root_dict = etree_to_dict(root)['coverage']
method_coverage = {} # {method_signature: (branch_rate, line_rate)}
for class_info in tqdm(dataset):
    for method_info in class_info:
        # if not f"{class_info.class_name}#{method_info.name}" in shorten_java_valid_map:
        #     continue

        count += 1
        source_packages: list = root_dict['project'][0]['package']
        # 找到focal class & focal method
        source_package = None
        for package in source_packages:
            if class_info.package_reference == package['@name']:
                source_package = package
                break
        assert source_package is not None

        source_files = source_package['file']
        source_file = None
        for file in source_files:
            if f"{class_info.class_name}.java" == file['@name']:
                source_file = file
                break
        assert source_file is not None

        source_report = source_file

        report_lines = source_report['line']
        branches = [line for line in report_lines if line['@type'] == 'cond']
        lines = [line for line in report_lines if line['@type'] == 'stmt']

        # 当测试某一个方法时，筛选出这个方法的line/branch
        branches = [branch for branch in branches if
                    method_info.start <= int(branch["@num"]) <= method_info.end]
        lines = [line for line in lines if
                 method_info.start <= int(line["@num"]) <= method_info.end]

        uncovered_branch = []
        false_uncovered_branch = []
        true_uncovered_branch = []
        unreachable_branch = []

        for branch in branches:
            if branch['@truecount'] == '0' or branch['@falsecount'] == '0':
                uncovered_branch.append(branch)
            if branch['@truecount'] != '0' and branch['@falsecount'] == '0':
                false_uncovered_branch.append(branch)
            if branch['@truecount'] == '0' and branch['@falsecount'] != '0':
                true_uncovered_branch.append(branch)
            if branch['@truecount'] == '0' and branch['@falsecount'] == '0':
                unreachable_branch.append(branch)



        branch_num_uncovered = len(false_uncovered_branch) + len(true_uncovered_branch) + 2 * len(unreachable_branch)
        branch_num_total = 1 if len(branches) == 0 else len(branches) * 2
        branch_num_covered = branch_num_total - branch_num_uncovered



        uncovered_line = []
        for line in lines:
            if line['@count'] == '0':
                uncovered_line.append(line)

        line_num_uncovered = len(uncovered_line)
        line_num_total = 1 if len(lines) == 0 else len(lines)
        line_num_covered = line_num_total - line_num_uncovered


        # if any([method_info.name.startswith(x) for x in ["get", "set", "is"]]) or method_info.name[0].isupper():
        #     branch_num_covered = branch_num_total
        #     line_num_covered = line_num_total

        if line_num_covered == 0:
            # count -= 1
            # continue
            branch_num_covered = 0

        branch_rate_tuple = (branch_num_covered, branch_num_total, branch_num_covered / branch_num_total)
        line_rate_tuple = (line_num_covered, line_num_total, line_num_covered / line_num_total)
        # print(f"Class: {class_info.class_name}, Method: {method_info.name}")
        # print(f"Branch Coverage: {branch_rate_tuple}")
        # print(f"Line Coverage: {line_rate_tuple}")
        method_coverage[f"{class_info.class_name}#{method_info.signature}"] = (branch_rate_tuple, line_rate_tuple)
        total_branch_tupe = (total_branch_tupe[0] + branch_rate_tuple[0], total_branch_tupe[1] + branch_rate_tuple[1])
        total_line_tupe = (total_line_tupe[0] + line_rate_tuple[0], total_line_tupe[1] + line_rate_tuple[1])
print(count)
print(f"Total Branch Coverage:  {total_branch_tupe} {total_branch_tupe[0] / total_branch_tupe[1]}")
print(f"Total Line Coverage: {total_line_tupe} {total_line_tupe[0] / total_line_tupe[1]}")

499it [00:00, 1073.61it/s]

5772
Total Branch Coverage:  (8378, 16525) 0.5069894099848714
Total Line Coverage: (14114, 28673) 0.4922400864925191
